<a href="https://colab.research.google.com/github/elvissoares/EQE595-SimMol/blob/main/notebooks/6_Soluto_e_Solvente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para usar o OpenMM no Google Colab devemos fazer o seguinte passo:
1. Instalar o `openmm[cuda12]` e o `mdtraj` via `pip`

In [ ]:
!pip install openmm[cuda12] mdtraj

2. Testando se a instalação deu certo e quais `Platform` estão disponíveis

In [ ]:
!python -m openmm.testInstallation

# Aula Prática 06 - Dinâmica Molecular de um Soluto em Água

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares)

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

Qual molécula iremos simular?

In [17]:
pdb = PDBxFile('1AKI.cif')

Condição termodinâmica

In [20]:
Temperatura = 298.15 * kelvin # Temperatura em Kelvin
Pressao = 1 * bar # Pressão em bar

Escolhendo os arquivos de campo de força `Amber19`

In [21]:
forcefield = ForceField('amber19-all.xml', 'amber19/tip3pfb.xml')

ValueError: Could not locate file "amber19-all.xml"

Cria a topolgia com moléculas de água ao redor (solvente)

In [ ]:
modeller = Modeller(pdb.topology, pdb.positions)

modeller.addSolvent(forcefield, model='tip3p', boxSize=Vec3(3.0, 3.0, 3.0)*nanometers)

PDBFile.writeFile(modeller.topology, modeller.positions, open(f'initial_solute.pdb', 'w'))

AttributeError: module 'openmm.app.forcefield' has no attribute 'createSystem'

In [ ]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=None)

# Plataforma (GPU se disponível)
platform = Platform.getPlatformByName('CUDA')
# Integrador
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 1*femtoseconds)

simulation = Simulation(modeller.topology, system, integrator,platform)
simulation.context.setPositions(modeller.positions)

Minimizando a energia

In [ ]:
simulation.minimizeEnergy()

energy = simulation.context.getState(getEnergy=True).getPotentialEnergy()
print(f'Energia: {energy}')


Definindo os relatórios

In [ ]:
# Remove reporters existentes, no caso dessa célula ser executada mais de uma vez.
simulation.reporters = []

# Escreve cada frame em arquivo DCD de trajetórias a cada 200 passos
simulation.reporters.append(DCDReporter(f"output_{my_molecule}.dcd", 100))

# Escreve num arquivo csv a cada 20 passos 
simulation.reporters.append(
    StateDataReporter(
        f"md_log_{my_molecule}.csv",
        100,
        step=True,
        time=True,
        potentialEnergy=True,
        totalEnergy=True,
        temperature=True,
        volume=True
    )
)

# Escreve na tela a cada 1000 passos 
simulation.reporters.append(
    StateDataReporter(stdout, 1000, step=True, totalEnergy=True,temperature=True,
        volume=True)
)

## Equilibração NVT

In [ ]:
simulation.context.setVelocitiesToTemperature(Temperatura)

simulation.step(5000)

## Produção NPT

In [ ]:
system.addForce(MonteCarloBarostat(Pressao, Temperatura))
simulation.context.reinitialize(preserveState=True)


print("Running NPT")
simulation.step(100_000)

# A próxima linha é necessária para Windows para fechar os arquivos antes de serem abertos
del simulation

## Visualizando a Simulação

In [ ]:
traj = mdtraj.load(f"output_{my_molecule}.dcd", top=f"initial_{my_molecule}.pdb")
view = nglview.show_mdtraj(traj, width='600px', height='400px')
view.clear_representations()
view.add_unitcell()
view.add_ball_and_stick(selection='not water')
#centralizado
view.center(selection='not water') # centraliza
view

## Analisando a estatística dos dados

In [ ]:
df = pd.read_csv(f"md_log_{my_molecule}.csv")

In [ ]:
df.head()

In [ ]:
df['Total Energy (kJ/mole)']

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True)

axs[0].plot(df['Time (ps)'],df['Potential Energy (kJ/mole)'],color='C0',label='U')
axs[0].legend(loc='best')
axs[0].set_ylabel('U (kJ/mole)')

axs[1].plot(df['Time (ps)'],df['Temperature (K)'],color='C3',label='T')
axs[1].legend(loc='best')
axs[1].set_ylabel('T (K)')

axs[2].plot(df['Time (ps)'],df['Box Volume (nm^3)'],color='C1',label='V')
axs[2].legend(loc='best')
axs[2].set_xlabel('Time (ps)')
axs[2].set_ylabel('Volume (nm$^3$)')

In [ ]:
timecut = 10
subsetT = df['Temperature (K)'][df['Time (ps)'] > timecut]
subsetU = df['Potential Energy (kJ/mole)'][df['Time (ps)'] > timecut]
subsetV = df['Box Volume (nm^3)'][df['Time (ps)'] > timecut]

In [ ]:
from IPython.display import display, Math

Umean = subsetU.mean()
sigmaU = subsetU.std()

display(Math(rf"U = {Umean:.2f} \pm {sigmaU:.2f}\ kJ/mole"))

In [ ]:
Tmean = subsetT.mean()
sigmaT = subsetT.std()

display(Math(rf"$T = {Tmean:.2f} \pm {sigmaT:.2f}\ K"))

In [ ]:
Vmean = subsetV.mean()
sigmaV = subsetV.std()

display(Math(rf"$V = {Vmean:.2f} \pm {sigmaV:.2f}\ nm^3"))

In [ ]:
fig, axs = plt.subplots(1, 3,figsize=(7,3))

axs[0].hist(subsetU,bins=30,density=True)
axs[0].set_xlabel(r'$U$ (kJ/mole)')

uarray = Umean+np.arange(-5*sigmaU,5*sigmaU,0.01*sigmaU)
axs[0].plot(uarray,np.sqrt(1/(2*np.pi*sigmaU**2))*np.exp(-0.5*(uarray-Umean)**2/sigmaU**2),color='k')

axs[1].hist(subsetT,bins=30,density=True,color='C3')
axs[1].set_xlabel(r'$T$ (K)')

Tarray = Tmean + np.arange(-5*sigmaT,5*sigmaT,0.01*sigmaT)
axs[1].plot(Tarray,np.sqrt(1/(2*np.pi*sigmaT**2))*np.exp(-0.5*(Tarray-Tmean)**2/sigmaT**2),color='k')

axs[2].hist(subsetV,bins=30,density=True,color='C2')
axs[2].set_xlabel(r'$V$ (nm$^3$)')

Varray = Vmean + np.arange(-5*sigmaV,5*sigmaV,0.01*sigmaV)
axs[2].plot(Varray,np.sqrt(1/(2*np.pi*sigmaV**2))*np.exp(-0.5*(Varray-Vmean)**2/sigmaV**2),color='k')

## Distâncias Interatômica

In [ ]:
distances = mdtraj.compute_distances(traj, [[0, 1], [1, 2]])
print(distances.shape)
print(distances[400, 0])

fig, ax = plt.subplots(num="dist")
for counter, col in enumerate(distances.T):
    ax.plot(df["Time (ps)"], col, "-", label=str(counter))
ax.set_xlabel("Tempo (ps)")
ax.set_ylabel("Distâncias (nm)")
ax.legend(loc=0)

## Análise de Raio de Giração 

In [ ]:
solute = traj.topology.select('not water')
rgyr = mdtraj.compute_rg(traj.atom_slice(solute))

plt.figure(figsize=(6, 4))
plt.plot(df['Time (ps)'],rgyr)
plt.xlabel("Time [ps]")
plt.ylabel('Raio de Giração (nm)')
plt.grid()

## Cálculo do Nº de Ligações de Hidrogênio

In [ ]:
hbonds = mdtraj.wernet_nilsson(traj)
numhbonds = [len(pairs) for pairs in hbonds]

fig, ax1 = plt.subplots(num="numh")
ax1.plot(df['Time (ps)'],numhbonds, label="Num. HBonds")
ax1.set_xlabel("Time [ps]")
ax1.set_ylabel("Number of hydrogen bonds", color="C0")

ax2 = ax1.twinx()
ax2.plot(df['Time (ps)'], df["Potential Energy (kJ/mole)"], color="C1", label="Pot. Energy")
ax2.set_ylabel("Potential energy [kJ/mol]", color="C1")

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

> Faça uma estimativa do custo computacional (tempo de simulação) no seu _hardware_ atual para simulações longas da ordem de 1ns. 
>